## CLIP(Contrastive Language-Image Pre-Training)

![CLIP_architecture](assets/clip_image1.png)

In [1]:
vision_model_name="openai/clip-vit-base-patch16"
text_model_name="t5-small"

In [2]:
from torchview import draw_graph
from transformers import ViTModel
import torch
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
model = ViTModel.from_pretrained("google/vit-base-patch16-224")
dummy_input = torch.randn(1, 3, 224, 224)
output = model.forward(dummy_input)
output.last_hidden_state.shape, output.pooler_output.shape

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(torch.Size([1, 197, 768]), torch.Size([1, 768]))

In [3]:
model

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (d

In [2]:
from datasets import load_dataset
flickr8k = load_dataset("jxie/flickr8k", data_dir="data")

In [6]:
for batch in flickr8k["train"]:
    image = batch["image"]
    text = batch["caption_0"]

KeyboardInterrupt: 

In [8]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained(text_model_name)
text_model = T5ForConditionalGeneration.from_pretrained(text_model_name)
tokenizer = T5Tokenizer.from_pretrained("t5-small")
source_text = "summarize: this is a long paragraph of text"
target_text = "short summary"
input_ids = tokenizer(source_text, return_tensors="pt").input_ids
labels = tokenizer(target_text, return_tensors="pt").input_ids

# 模型前向传播（训练用）
outputs = text_model(input_ids=input_ids, 
                     labels=labels,  
                     output_hidden_states=True,
                     return_dict=True)
encoder_last_hidden = outputs.encoder_hidden_states[-1]  # [1, src_len, 512]
pooled = encoder_last_hidden.mean(dim=1)
pooled.squeeze().shape

torch.Size([512])